## Aula 03 - Filtragem Baseada em Conteúdo - Exercícios

In [2]:
import pandas as pd
import numpy as np

### Importar base de dados

In [4]:
import wget
!python -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz


Saved under ml-20m-compact.tar (16).gz


x dataset/
x dataset/tags_sample.csv
x dataset/._.DS_Store
x dataset/.DS_Store
x dataset/movies_sample.csv
x dataset/._genome-tags.csv
x dataset/genome-tags.csv
x dataset/._ml-youtube.csv
x dataset/ml-youtube.csv
x dataset/._genome-scores.csv
x dataset/genome-scores.csv
x dataset/ratings_sample.csv


In [5]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [6]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [7]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.movieId] = row.title


In [8]:
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,18,34,Gregory Peck,1329962459
1,18,34,need to own,1329962471
2,18,34,romantic comedy,1329962476
3,18,34,Rome,1329962490
4,18,34,royalty,1329962474


### Divisão da base em treino e teste

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

***Exercício 01:*** Aplique a filtragem baseada em conteúdo (ItemAttributeKNN do CaseRecommender) com as tags associadas aos filmes da base. Utilize Jaccard como métrica de similaridade e k=5 vizinhos para predição.

Documentação do ItemAttributeKNN: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [12]:
# TODO

from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

# Save the train, test and movies_tags dfs to files as csv
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')
movies_tags[['movieId', 'tag']].to_csv('items_tags.dat', index=False, sep='\t', header=False)

# Run the model
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_tags.dat', output_file='output1.dat', k_neighbors=5, similarity_metric='jaccard', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 6.902027 sec
>> metadata:: 231 items and 1979 metadata (6274 interactions) | sparsity:: 98.63%
prediction_time:: 1.037632 sec
Eval:: MAE: 0.736001 RMSE: 0.971392 


In [13]:
# Get the predictions result output file
predictions = pd.read_csv('output1.dat', sep='\t', header=None, names=['userId', 'movieId', 'predRating'])

# Merge the original df and predictions on userId and movieId to compare
merged_df = pd.merge(df, predictions, on=['userId', 'movieId'], how='inner')

# Print a sample from the merged dataframe
merged_df.sample(5)

,userId,movieId,rating,title,predRating
36744,10697,5,3.0,"I, Robot (2004)",3.189875
6430,1869,8,2.5,"Karate Kid, The (1984)",3.128377
24019,7007,18,5.0,Life Is Beautiful (La Vita è bella) (1997),4.099691
28034,8169,84,3.5,National Treasure: Book of Secrets (2007),3.423931
22015,6399,27,3.0,Zero Effect (1998),3.274535


### Preparação para o exercício 2 - Download e extração de metadados multimídia

In [15]:
!python -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-features.tar.gz
! tar -xvzf ml-20m-features.tar.gz


Saved under ml-20m-features.tar (12).gz


x features/
x features/._m4infus_max_histogram_300_sn.arq
x features/m4infus_max_histogram_300_sn.arq
x features/._mm_avg_histogram_100_sn.arq
x features/mm_avg_histogram_100_sn.arq
x features/._visual_histogram_100_sn.arq
x features/visual_histogram_100_sn.arq
x features/._visual_histogram_50_sn.arq
x features/visual_histogram_50_sn.arq
x features/._aural_histogram_50.arq
x features/aural_histogram_50.arq
x features/._mm_max_histogram_300.arq
x features/mm_max_histogram_300.arq
x features/._m4infus_max_histogram_50.arq
x features/m4infus_max_histogram_50.arq
x features/._mm_max_histogram_100.arq
x features/mm_max_histogram_100.arq
x features/._mm_max_histogram_50_sn.arq
x features/mm_max_histogram_50_sn.arq
x features/._visual_histogram_100.arq
x features/visual_histogram_100.arq
x features/._visual_histogram_300.arq
x features/visual_histogram_300.arq
x features/._aural_histogram_100_sn.arq
x features/aural_histogram_100_sn.arq
x features/._mm_avg_histogram_100.arq
x features/mm_avg_

In [16]:
import pickle

with open('./features/visual_histogram_50.arq', 'rb') as arq_visualHistograms:
    visualHistograms = pickle.load(arq_visualHistograms)
print('No. movies: ' + str(len(visualHistograms)))
print('Size of each word: ' + str(len(visualHistograms[0])))

No. movies: 433
Size of each word: 50


In [17]:
visualHistograms[0]

array([0.00948992, 0.00237248, 0.00355872, 0.01304864, 0.05338078,
       0.00711744, 0.05456702, 0.00355872, 0.0059312 , 0.00355872,
       0.0059312 , 0.00474496, 0.00711744, 0.04507711, 0.07591934,
       0.00355872, 0.1316726 , 0.05338078, 0.00830368, 0.        ,
       0.01304864, 0.00474496, 0.02253855, 0.00355872, 0.02728351,
       0.00237248, 0.02016607, 0.00830368, 0.0059312 , 0.00237248,
       0.00355872, 0.04033215, 0.01067616, 0.00237248, 0.00830368,
       0.05931198, 0.0059312 , 0.01067616, 0.02965599, 0.01423488,
       0.00711744, 0.02609727, 0.        , 0.00237248, 0.01423488,
       0.03084223, 0.02016607, 0.00118624, 0.08778173, 0.02253855])

***Exercício 02:*** Como visto, o algoritmo ItemAttributeKNN pode ser usado com diferentes tipos de metadados, como gêneros, tags e palavras no geral. Mais do que isso, podemos adaptá-lo também para que a similaridade entre itens seja feita com base em informações multimídia, como imagens, áudio, etc. 

A base de dados utilizada até o momento, ml-20m-compact.tar.gz possui, além das interações de usuários com filmes, uma série de arquivos que contém informações multimídia que foram extraídas dos trailers de cada filme. Esses arquivos estão condensados no zip ml-20m-features.tar.gz, o qual foi feito o download e extraído acima. 

Considere por exemplo o arquivo visual_histogram_50.arq. Ele possui 433 vetores (no. de filmes) de tamanho 50. Podemos pensar que cada vetor desse representa informações visuais (cor, brilho, imagem, etc.) que foram extraídas dos trailers de cada filme. 

Sua tarefa é usar esses vetores de características visuais no cálculo de similaridade entre os filmes, e em seguida, aplicar essas similaridades no algoritmo ItemAttributeKNN para gerar recomendações. 

Dica 1: para calcular a similaridade entre dois vetores pode-se usar o ângulo de cosseno (vide https://en.wikipedia.org/wiki/Cosine_similarity). 

Dica 2: é possível passar para o algoritmo ItemAttributeKNN a matriz de similaridade entre itens, por meio do parâmetro similarity_file=arquivo. Veja em: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [19]:
# TODO
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the similarity using the cosine_similarit from sklearn
similarity_matrix = cosine_similarity(visualHistograms)

# Save the similarity_matrix as a file 
np.savetxt('similarity_matrix.arq', similarity_matrix)

# Run the model using the similarity_matrix
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_tags.dat', output_file='output2.dat', k_neighbors=5, similarity_file='similarity_matrix.arq', as_similar_first=True).compute()


[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 6.408458 sec
>> metadata:: 231 items and 1979 metadata (6274 interactions) | sparsity:: 98.63%
prediction_time:: 0.552928 sec
Eval:: MAE: 0.729791 RMSE: 0.959396 


In [20]:
# Get the predictions result output file
predictions = pd.read_csv('output2.dat', sep='\t', header=None, names=['userId', 'movieId', 'predRating'])

# Merge the original df and predictions on userId and movieId to compare
merged_df = pd.merge(df, predictions, on=['userId', 'movieId'], how='inner')

# Print the first 5 rows from the merged dataframe
merged_df.sample(5)

,userId,movieId,rating,title,predRating
7556,2191,19,3.5,Reservoir Dogs (1992),4.001801
36760,10702,14,5.0,Go (1999),3.647070
819,246,103,3.0,Red (2010),2.956513
20877,6056,22,5.0,Natural Born Killers (1994),4.203375
11092,3248,129,4.0,Interstellar (2014),3.613136


***Exercício 03:*** Implementar uma função que retorna a probabilidade de um item ser relevante para um usuário, considerando os gêneros dos filmes. Utilize métodos probabilísticos. 
- A partir do conjunto de treinamento, obter todas as interações do usuário u.
- Rotular as notas desse usuário como: item relevante se nota >=3 e não relevante se nota < 3. 
- Dado um item do conjunto de teste, aplicar o Teorema de Bayes com suavização de Laplace. Utilizar os gêneros associados. 
- Retornar se o item é ou não relevante, e em seguida, comparar o resultado com a nota real que esse usuário deu para o item (disponível no conjunto de teste).

In [22]:
from collections import defaultdict

# Create a new df so each movie has each genre in separated rows
movies_genres = movies.drop('genres', axis=1).join(movies.genres.str.split('|', expand=True)
             .stack().reset_index(drop=True, level=1).rename('genre'))

# Merge the train df and the movies_genres df to add the genre of the movies
df_merged = pd.merge(train, movies_genres, on=['title'])

# Add a column that is 1 if rating >= 3 or 0 if not
df_merged['relevant'] = (df_merged['rating'] >= 3).astype(int)

# Function that calculates if a movie is relevant or not to a specific user, if
# the probability is larger than 0.5, then its relevant
def calculate_relevance_probability(user_id, movie_id, df_merged, test_df):
    # Get the interactions of the specific user 
    user_interactions = df_merged[df_merged['userId'] == user_id]
    
    # Get the genres of the specific movie
    movie_genres = df_merged[df_merged['movieId_x'] == movie_id]['genre'].unique().tolist()
   
    total_relevant = 0
    total_not_relevant = 0
    genre_count_relevant = defaultdict(int)
    genre_count_not_relevant = defaultdict(int)
    
    # Label movies as relevant (rating >= 3) or not relevant (rating < 3)
    for _, row in user_interactions.iterrows():
        genres = row['genre']
        relevant = row['relevant']
        
        if relevant:
            total_relevant += 1
            for genre in genres:
                genre_count_relevant[genre] += 1
        else:
            total_not_relevant += 1
            for genre in genres:
                genre_count_not_relevant[genre] += 1
    
    # Total number of unique genres
    unique_genres = set(genre_count_relevant.keys()).union(set(genre_count_not_relevant.keys()))
    total_unique_genres = len(unique_genres)
    
    # Calculate probabilities using Laplace smoothing
    prob_relevant = total_relevant / (total_relevant + total_not_relevant)
    prob_not_relevant = total_not_relevant / (total_relevant + total_not_relevant)
    
    prob_genre_given_relevant = 1
    prob_genre_given_not_relevant = 1
    
    for genre in movie_genres:
        # Probability of seeing the genre given the movie is relevant or not, with Laplace smoothing
        prob_genre_given_relevant *= (genre_count_relevant[genre] + 1) / (total_relevant + total_unique_genres)
        prob_genre_given_not_relevant *= (genre_count_not_relevant[genre] + 1) / (total_not_relevant + total_unique_genres)
    
    # Calculate the final probability using Bayes' Theorem
    final_prob_relevant = prob_genre_given_relevant * prob_relevant
    final_prob_not_relevant = prob_genre_given_not_relevant * prob_not_relevant
    
    # Normalize to get the probability
    total_prob = final_prob_relevant + final_prob_not_relevant
    if total_prob == 0:
        return 0  # To handle division by zero if both probabilities are zero
    final_prob = final_prob_relevant / total_prob
    
    # Determine if the movie is relevant
    is_relevant = final_prob >= 0.5
    
    # Get the actual rating for comparison from the test df
    actual_rating = test_df[(test_df['userId'] == user_id) & (test_df['movieId'] == movie_id)]['rating']
    actual_rating = actual_rating.values[0] if not actual_rating.empty else None

    print(f"O filme {merged_df[merged_df['movieId'] == movie_id]['title'].values[0]} é classificado como {'relevante' if is_relevant else 'não relevante'} para o usuário {user_id}.")
    if actual_rating is not None:
        print(f"A nota real dada pelo usuário foi: {actual_rating}")
    else:
        print("Nota real não disponível.")
            

calculate_relevance_probability(1464, 19, df_merged, test)
calculate_relevance_probability(1836, 60, df_merged, test)
calculate_relevance_probability(7518, 24, df_merged, test)
calculate_relevance_probability(6472, 112, df_merged, test)
calculate_relevance_probability(1165, 3, df_merged, test)

O filme Reservoir Dogs (1992) é classificado como relevante para o usuário 1464.
A nota real dada pelo usuário foi: 4.5
O filme She's the Man (2006) é classificado como relevante para o usuário 1836.
A nota real dada pelo usuário foi: 2.5
O filme Deep End of the Ocean, The (1999) é classificado como relevante para o usuário 7518.
A nota real dada pelo usuário foi: 3.0
O filme Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965) é classificado como não relevante para o usuário 6472.
A nota real dada pelo usuário foi: 1.5
O filme American Psycho (2000) é classificado como não relevante para o usuário 1165.
A nota real dada pelo usuário foi: 2.0


***Exercício 04:*** No notebook de exemplos, existe uma implementação de Filtragem Baseada em Conteúdo usando Multi-Layer Perceptron como um regressor (MLPRegressor) que prevê a nota de cada usuário para filmes ainda não vistos. O algoritmo retorna uma lista de top K filmes com maiores notas. O treinamento é realizado utilizando as notas que o usuário deu para os filmes e seus respectivos gêneros.
- Usando a classe MLPClassifier (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html), implemente uma versão que classifica os filmes não vistos como relevante ou não-relevante.
- No conjunto de dados, realize a binarização das notas, de modo que notas acima de 3 são relevantes e notas abaixo de 3 são não-relevantes.
- Retorne os top k filmes mais relevantes.
- Para um usuário qualquer da base (ou algum outro usuário fictício), analise subjetivamente a qualidade das recomendações, comparando os modelos MLPRegressor e MLPClassifier. 

In [24]:
genres_unique = pd.DataFrame(movies.genres.str.split('|').tolist()).stack().unique()
genres_unique = pd.DataFrame(genres_unique, columns=['genre'])
movies_exp = movies.join(movies.genres.str.get_dummies().astype(bool))
movies_exp.drop('genres', inplace=True, axis=1)

In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelBinarizer

# Initialize objects to store the models
user_models_mlp_class = {}

# For each user, train a model
for user_id in train['userId'].unique():
    # Get the movies rated by the user
    user_ratings = train[(train['userId'] == user_id)]

    # Get the movies rated by the user with the genres
    user_movies = pd.merge(user_ratings, movies_exp, on='title')

    if len(user_movies) > 1:
        # Prepare data for MLP training
        X = user_movies.drop(['userId', 'movieId_x', 'movieId_y', 'rating', 'title'], axis=1)
        
        # Binzarize the ratings (relevant (1) if rating >= 3, not-relevant (0) if rating < 3)
        y = (user_movies['rating'] >= 3).astype(int)

        # Train MLPClassifier
        mlp_class = MLPClassifier(hidden_layer_sizes=(50, 30), activation='relu', max_iter=1000, random_state=42)
        mlp_class.fit(X, y)

        # Save the model for each user
        user_models_mlp_class[user_id] = mlp_class


def recommend_movies(user_id, k=10):
    if user_id in user_models_mlp_class:
        # Get the user model
        mlp_class = user_models_mlp_class[user_id]
        
        # Predict the relevance for all movies
        all_movies = movies_exp.drop(['movieId', 'title'], axis=1)
        predicted_probabilities = mlp_class.predict_proba(all_movies)[:, 1]  # Probabilidade de ser relevante (classe 1)

        # Add the predictions to the original df
        movies_exp['predicted_relevance'] = predicted_probabilities

        # Remove the movies that the user already rated
        user_rated_movies = train[train['userId'] == user_id]['movieId']
        recommended_movies = movies_exp[~movies_exp['movieId'].isin(user_rated_movies)]

        # Order the movies by predict relevance and show the top k
        recommended_movies = recommended_movies.sort_values(by='predicted_relevance', ascending=False)

        return recommended_movies[['title', 'predicted_relevance']].head(k)
    else:
        return f"User {user_id} does not have enough data to train a model."

# Test for user 20
top_movies = recommend_movies(20, k=10)
print("Top 10 filmes recomendados:")
print(top_movies)

Top 10 filmes recomendados:
                                                 title  predicted_relevance
12   Alphaville (Alphaville, une étrange aventure d...             1.000000
240                                     Balance (1989)             1.000000
194                                  Quiet, The (2005)             0.999999
136                                       Frogs (1972)             0.999999
162                                   Boogeyman (2005)             0.999999
216                              Happening, The (2008)             0.999998
229                             Hangover Square (1945)             0.999998
190                         Seventh Victim, The (1943)             0.999997
31                            Conspiracy Theory (1997)             0.999996
100           Horror Planet (a.k.a. Inseminoid) (1981)             0.999996
